# GPT Recipe Extractor

## Imports

In [1]:
import base64
import requests
import os
import json
import re
from pathlib import Path
import time

In [3]:
# load our key
api_key = open("key.txt", "r").read().strip("\n")

## Configs

In [4]:
# Source and destination folders
source_folder_single_page = 'input_images_single'
source_folder_multi_page = 'input_images_multi'
destination_folder = 'output_markdown'

MAX_TOKENS = 4096
max_retries = 5

In [5]:
extraction_prompt = """
Attached is a magazine page containing one or more recipes. Please extract the following for each recipe in markdown format:
-Recipe name (with Heading level 1)
-Preparation time (with Heading level 2)
-Servings (with Heading level 2)
-Ingredients (with Heading level 2)
-Instructions (with Heading level 2)
-Cuisine name inferred from recipe (with Heading level 2)
-Tips (with Heading level 2)
"""

## Shared Functions

In [6]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function to save markdown to file
def save_markdown(markdown, destination_folder):
    # Split the markdown into individual recipes
    recipes = re.split(r'\n# ', markdown)

    for i, recipe in enumerate(recipes):
        # Add back the removed heading
        if i > 0:
            recipe = '# ' + recipe

        # Extract the heading from the recipe
        heading = re.search(r'^# (.*)$', recipe, re.MULTILINE)
        if heading:
            filename = heading.group(1) + '.md'
        else:
            filename = f'default{i}.md'
        
        # Save the recipe to a file
        with open(os.path.join(destination_folder, filename), 'w') as f:
            f.write(recipe)

## Process Single Page Recipes

In [13]:
# Get a list of all image files in the source directory
image_files = [f for f in os.listdir(source_folder_single_page) if f.endswith('.jpg')]

for image_file in image_files:
    # Getting the base64 string
    base64_image = encode_image(os.path.join(source_folder_single_page, image_file))

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": extraction_prompt
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/jpeg;base64,{base64_image}"
                }
              }
            ]
          }
        ],
        "max_tokens": MAX_TOKENS
    }

    for i in range(max_retries):
        try:
            response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
            response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx

            # Extract the markdown from the response and save it to a file
            markdown = json.loads(response.text)['choices'][0]['message']['content']
            save_markdown(markdown, destination_folder)
            break  # Break the loop if the request is successful
        except requests.exceptions.RequestException:
            wait_time = 2 ** i  # Exponential backoff
            print(f"Request failed. Waiting for {wait_time} seconds before retrying...")
            time.sleep(wait_time)
    else:
        print(f"Failed to process image {image_file} after {max_retries} retries.")

Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 seconds before retrying...
Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 seconds before retrying...
Request failed. Waiting for 4 seconds before retrying...
Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 seconds before retrying...
Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 seconds before retrying...
Request failed. Waiting for 4 seconds before retrying...
Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 seconds before retrying...
Request failed. Waiting for 4 seconds before retrying...
Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 seconds before retrying...
Request failed. Waiting for 4 seconds before retrying...
Request failed. Waiting for 1 seconds before retrying...
Request failed. Waiting for 2 s

## Process Multiple Page Recipes

In [7]:
import glob

# Get a list of all subfolders in the source directory
recipe_folders = [f.path for f in os.scandir(source_folder_multi_page) if f.is_dir()]

for recipe_folder in recipe_folders:
    # Get a list of all image files in the recipe subfolder
    image_files = glob.glob(os.path.join(recipe_folder, '*.jpg'))

    # Prepare the messages for the API request
    messages = [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": extraction_prompt
            },
        ]
    }]
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    for image_file in image_files:
        # Getting the base64 string
        base64_image = encode_image(image_file)

        # Add the image to the messages
        messages[0]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": MAX_TOKENS
    }

    for i in range(max_retries):
        try:
            response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
            response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx

            # Extract the markdown from the response and save it to a file
            markdown = json.loads(response.text)['choices'][0]['message']['content']
            save_markdown(markdown, destination_folder)
            break  # Break the loop if the request is successful
        except requests.exceptions.RequestException:
            wait_time = 2 ** i  # Exponential backoff
            print(f"Request failed. Waiting for {wait_time} seconds before retrying...")
            time.sleep(wait_time)
    else:
        print(f"Failed to process image {image_file} after {max_retries} retries.")